In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [2]:
# VARIABLES TO UPDATE FOR EACH SHAKEUP:

TRANSLATION_FILE = 'DecemberServiceChanges_ForTranslation_110722.xlsx'
TRANLSATION_LANGUAGES = {
    'ENGLISH': 'en',
    'SPANISH': 'es',
    'CHINESE': 'zh-tw',
    'KOREAN': 'ko',
    'VIETNAMESE': 'vi',
    'JAPANESE': 'ja',
    'RUSSIAN': 'ru',
    'ARMENIAN': 'hy'
}

INTRO_ROWS = [0, 1, 2]

SUMMARY_START = 3
SUMMARY_END = 7

DETAIL_START = 8

GTFS_ROUTES_URL = 'https://gitlab.com/LACMTA/gtfs_bus/-/raw/master/routes.txt'

Grab the Excel file with all the translations of the TakeOne, and put it in the `_data` folder, renamed to `allChanges.xlsx`

In [3]:
# pandas uses the xlrd package by default, but xlrd has removed support for xlsx files
# thus we need to install and use openpyxl and set that to the engine used by read_excel()
source_df = pd.read_excel(TRANSLATION_FILE, engine = 'openpyxl')
source_df.rename(columns=lambda x: x.strip())


# source_df.columns = source_df.columns.str.replace(' ', '')
source_df = source_df.rename(columns = TRANLSATION_LANGUAGES)

#source_df

Create a dataframe with a subset of the data (just the line change details).

Manually set the range of rows to copy.

In [4]:
line_changes_df = source_df.iloc[DETAIL_START:]

# Clean up by resetting the index and removing NaN rows
line_changes_df = line_changes_df.reset_index(drop=True)
line_changes_df = line_changes_df.dropna(subset=['en'])

#line_changes_df

Manually adjust the data as needed.

In [5]:
# Combine the last two rows: 55, 56
# The row indices are different when accessing them as a Series versus the DataFrame

# for column in line_changes_df:
#     colSeries = line_changes_df[column]
#     combinedText = colSeries.values[37] + colSeries.values[38]
#     line_changes_df.at[55, colSeries.name] = combinedText
#     line_changes_df.at[56, colSeries.name] = ''

# line_changes_df = line_changes_df.drop([56])

# line_changes_df = line_changes_df.reset_index(drop=True)

# line_changes_df

In [6]:
# Get the line number label by splitting based on the dash or em-dash
# Create a new column 'label' that uses the first value from the split
# Note that this value may be a line number or may be a text description

# em-dash: –
# normal dash: -

line_changes_df['label'] = line_changes_df['en'].str.split('-').str[0].str.strip()

#line_changes_df

In [7]:
# Remove line numbers from details

for column in line_changes_df:
    for i in range(len(line_changes_df[column])):
        if '՝' in line_changes_df.at[i, column]:
            content = "՝".join(line_changes_df.at[i, column].split('՝')[1::])
            line_changes_df.at[i, column] = content
        elif ':' in line_changes_df.at[i, column]:
            content = ":".join(line_changes_df.at[i, column].split(':')[1::])
            line_changes_df.at[i, column] = content
        elif '–' in line_changes_df.at[i, column]:
            content = "-".join(line_changes_df.at[i, column].split('–')[1::])
            line_changes_df.at[i, column] = content
        elif '-' in line_changes_df.at[i, column]:
            content = "-".join(line_changes_df.at[i, column].split('-')[1::])
            line_changes_df.at[i, column] = content

# line_changes_df.count()

In [8]:
# Duplicate rows that have two line numbers (based on the label column)

def separate_rows(df, col, delim):
    colSeries = df[col]
    for item in colSeries:
        if delim in str(item):
            routes = item.split(delim)
            matching_df = df.loc[df[col] == item]

            matching_df = matching_df.append(matching_df)
            matching_df = matching_df.reset_index(drop=True)
            
            matching_df.at[0, col] = routes[0].strip()
            matching_df.at[1, col] = routes[1].strip()

            df = df[df[col] != item]
            df = df.append(matching_df)
            df = df.reset_index(drop=True)
    return df

line_changes_df = separate_rows(line_changes_df, 'label', '/')

#line_changes_df.count()

In [9]:
# Create a new column 'line' that uses the value from column 'label' only if it is a digit
def check_digit(x):
    if x['label'].isdigit():
        return x['label']
    else:
        return 0

line_changes_df['line'] = 0
line_changes_df['line'] = line_changes_df.apply(check_digit, axis=1)

# Set specific instance of the C & K Line Link, which is actually route number 857

line_changes_df.loc[line_changes_df.label == 'C & K Line Link', 'line'] = 857

# line_changes_df

In [10]:
# Create an ordered list of routes based on the current GTFS routes.txt file.  Combined lines are separated into their own rows
routes_df = pd.read_csv(GTFS_ROUTES_URL, usecols=['route_short_name'], dtype={'route_short_name':str})

In [11]:

split_routes_df = separate_rows(routes_df, 'route_short_name', '/')
split_routes_df = split_routes_df.dropna()

In [12]:
index_names = split_routes_df[(split_routes_df['route_short_name'].str.contains('Stadium'))].index

routes_df = split_routes_df.drop(index_names) # drop the stadium routes
routes_df = routes_df.astype({'route_short_name':'int'}) # cast to int

# add back in the BRT lines since they don't have values for route_short_name
routes_df = routes_df.append({'route_short_name':854}, ignore_index=True)
routes_df = routes_df.append({'route_short_name':901}, ignore_index=True)
routes_df = routes_df.append({'route_short_name':910}, ignore_index=True)
routes_df = routes_df.append({'route_short_name':950}, ignore_index=True)

routes_df = routes_df.sort_values(by='route_short_name').reset_index(drop=True) # sort routes in ascending order and reset the index
routes_df = routes_df.reset_index() # create a new column with the reset index values

# 'route_short_name' contains all routes separated
# 'index' contains the number ordering of the routes

# recast route_short_name to string for joining
routes_df = routes_df.astype({'route_short_name':'str'})

# rename columns

routes_df = routes_df.rename(columns={'index':'order','route_short_name':'line'})

# routes_df

In [13]:
# Not using this as of 11/22/2022
# Read in full list of lines as pulled from the current GTFS routes.txt file (slightly modified)

# routes_df = pd.read_csv('routes.csv', dtype={'line': str, 'order': int})
# routes_df['line'] = routes_df['line'].astype(str)
# routes_df['line'].apply(lambda x: x.strip())

routes_df.count()

order    119
line     119
dtype: int64

In [14]:
# Merge the two dataframes using an outer join

line_changes_df['line'] = line_changes_df['line'].astype(str)
routes_and_line_changes_df = pd.merge(line_changes_df, routes_df, on='line', how='outer')

routes_and_line_changes_df = routes_and_line_changes_df.sort_values(by='order')

# confirm we have the correct final counts of lines with changes and total number of lines
routes_and_line_changes_df = routes_and_line_changes_df.reset_index(drop=True)
# routes_and_line_changes_df

In [15]:
# check to see if any lines have no order value attached
routes_and_line_changes_df.loc[routes_and_line_changes_df['order'].isna()]

,en,es,zh-tw,hy,ko,vi,ja,ru,label,line,order


In [16]:
# add the PDFs
import os

#define the folders to look through
# folders = os.listdir("../files/schedules")

#set an array for the file types
timetables_list = []

# from javascript file `rename.js`:
# let newFileName = '';

# let match = file.match(/^([0-9]*)( |_|-)*([0-9]*)( |_|-)*(TT)( |_|-)*([0-9]*)( |_|-)*([0-9]*)( |_|-)*([0-9]*)( |_|-)*.pdf$/);

# if (match[3] == '') {
#     newFileName = `${match[1]}_TT_${match[7]}-${match[9]}-${match[11]}.pdf`;
# } else {
#     newFileName = `${match[1]}-${match[3]}_TT_${match[7]}-${match[9]}-${match[11]}.pdf`;
# }

# if (file != newFileName) {
#     renameSync(join(dir, file), join(dir, newFileName));
#     console.log(`Renamed:\n${file}\nto\n${newFileName}\n`);
# }

#create a list of file types
for root, dirs, files in os.walk("../files/schedules"):
    for filename in files:

        lines = filename.replace(" ","_").split("_TT")[0].split("-")
        for line in lines:
            this_schedule = {}
            this_schedule['line'] = line.lstrip("0")
            this_schedule['newSchedule'] = "/files/schedules/"+filename
            timetables_list.append(this_schedule)

schedule_df = pd.DataFrame(timetables_list)
# schedule_df.tail(10)

In [17]:
# merge schedules with list

new_pdfs_and_routes_and_line_changes_df = pd.merge(routes_and_line_changes_df, schedule_df, on='line', how='outer')
# new_pdfs_and_routes_and_line_changes_df.tail(20)

# September 2022 shakeup: the 134 is a new line.  the 807 is the new crenshaw line
# December 2022 shakeup: the 489 is missing from the GTFS routes and the 802 is the B Line (Red)

In [18]:
# pull current schedule PDFs from the metro.net WP REST API

wp_url = 'https://www.metro.net/wp-json/wp/v2/line-override'
wp_df = pd.read_json(wp_url)

wp_df = pd.DataFrame(wp_df['acf'])
# wp_df


In [19]:
# Grab the line-override entries from WordPress that have a pdf_file.url value

def get_urls(x):
    temp_df = pd.json_normalize(x['acf'])
    if 'pdf_file.url' in temp_df.columns:
        return temp_df['pdf_file.url']
    return

wp_df_urls = wp_df.apply(get_urls, axis=1)

wp_df_urls = wp_df_urls.dropna()

wp_df_urls = wp_df_urls.rename(columns={0: "currentSchedule"})

# wp_df_urls

In [20]:
# check for any WordPress Line Overrides with no urls

def get_no_urls(x):
    temp_df = pd.json_normalize(x['acf'])
    if 'pdf_file.url' not in temp_df.columns:
        return temp_df['line_id']
    return

wp_df_no_urls = wp_df.apply(get_no_urls, axis=1)

wp_df_no_urls = wp_df_no_urls.dropna()

wp_df_no_urls

Series([], dtype: object)

In [21]:
# Grab the line and add it to a separate column

def get_lines(x):
    split_url = x['currentSchedule'].split('/')
    filename = split_url[len(split_url)-1].replace(' ', '_')
    line = ''

    if '_-' in filename:
        filename = filename.replace('_-', '_')

    if '_TT' in filename:
        line = filename.split('_TT')[0].lstrip('0')
    elif '-TT' in filename:
        line = filename.split('-TT')[0].lstrip('0')
    elif '.pdf' in filename:
        line = filename.split('_')[0].lstrip('0')
    else:
        line = filename
    x['line'] = line
    return

current_pdfs_list = []

wp_df_urls['line'] = ''
wp_df_urls.apply(get_lines, axis=1)

# wp_df_urls

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
30     None
31     None
32     None
33     None
34     None
35     None
36     None
37     None
38     None
39     None
40     None
41     None
42     None
43     None
44     None
45     None
46     None
47     None
48     None
49     None
50     None
51     None
52     None
53     None
54     None
55     None
56     None
57     None
58     None
59     None
60     None
61     None
62     None
63     None
64     None
65     None
66     None
67     None
68     None
69     None
70     None
71     None
72     None
73     None
74     None
75     None
76     None
77     None
78     None
79     None
80     None
81     None
82     None
83  

In [22]:
# wp_df_urls = separate_rows(wp_df_urls, 'line', '_')

wp_df_urls = wp_df_urls.drop_duplicates(subset=['line'])

colSeries = wp_df_urls['line']

for item in colSeries:
    if '-' in item:
        routes = item.split('-')
        matching_df = wp_df_urls.loc[wp_df_urls['line'] == item]
        matching_df = matching_df.append(matching_df)
        matching_df = matching_df.reset_index(drop=True)

        matching_df.at[0, 'line'] = routes[0].strip().lstrip('0')
        matching_df.at[1, 'line'] = routes[1].strip().lstrip('0')

        wp_df_urls = wp_df_urls[wp_df_urls['line'] != item]
        wp_df_urls = wp_df_urls.append(matching_df)
        wp_df_urls = wp_df_urls.reset_index(drop=True)

# wp_df_urls


In [23]:
# combine current schedules 

details_df = pd.merge(new_pdfs_and_routes_and_line_changes_df, wp_df_urls, on='line', how='left')

# details_df

In [24]:
# check for any lines that are missing a currentSchedule
# this would indicate a problem in WordPress that needs to be fixed

details_df.loc[details_df['currentSchedule'].isna()]


,en,es,zh-tw,hy,ko,vi,ja,ru,label,line,order,newSchedule,currentSchedule


In [25]:
# October 2022 shakeup: manual adjustment for line 534, which is being renamed to the 134

# details_df.loc[details_df['line'] == '534', 'newSchedule'] = details_df.loc[details_df['line'] == '134', 'newSchedule'].values[0]

# details_df.loc[details_df['line'] == '534']

# details_df = details_df[details_df['line'] != '134']

# details_df.tail(10)

In [26]:
# manual filling in of label field

def apply_line_labels(row):
    # if pd.isna(details_df.at[row.name, 'label']):
    line_value = details_df.at[row.name, 'line']

    if '802' in line_value:
        details_df.at[row.name, 'label'] = 'B Line'           
    elif '807' in line_value:
        details_df.at[row.name, 'label'] = 'K Line'
    elif '854' in line_value:
        details_df.at[row.name, 'label'] = 'L Line Shuttle'
    elif '857' in line_value:
        details_df.at[row.name, 'label'] = 'C & K Line Link'
    elif '901' in line_value:
        details_df.at[row.name, 'label'] = 'G Line'
    elif '910' in line_value:
        details_df.at[row.name, 'label'] = 'J Line (910)'
    elif '950' in line_value:
        details_df.at[row.name, 'label'] = 'J Line (950)'
    else:
        details_df.at[row.name, 'label'] = line_value
    return

details_df.apply(apply_line_labels, axis=1)

# details_df.tail(10)

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
30     None
31     None
32     None
33     None
34     None
35     None
36     None
37     None
38     None
39     None
40     None
41     None
42     None
43     None
44     None
45     None
46     None
47     None
48     None
49     None
50     None
51     None
52     None
53     None
54     None
55     None
56     None
57     None
58     None
59     None
60     None
61     None
62     None
63     None
64     None
65     None
66     None
67     None
68     None
69     None
70     None
71     None
72     None
73     None
74     None
75     None
76     None
77     None
78     None
79     None
80     None
81     None
82     None
83  

In [27]:
# Sort based on line number
# Must cast the line column to int for sorting
details_df = details_df.astype({'line':int})
details_df = details_df.sort_values(by='line').reset_index(drop=True)
details_df = details_df.astype({'line':str})
# details_df.tail(10)

In [28]:
summary_df = source_df.loc[SUMMARY_START:SUMMARY_END].reset_index(drop=True)
# summary_df

In [29]:
# will need to customize for each shakeup

for column in summary_df:
    colSeries = summary_df[column]

    combinedText = '<p>' + colSeries.values[1] + '</p><p>' + colSeries.values[2] + '</p><p>' + colSeries.values[3] + '</p>'
    summary_df.at[1, colSeries.name] = combinedText

summary_df = summary_df.drop([2, 3])
summary_df = summary_df.reset_index(drop=True)
summary_df.at[0, 'type'] = 'description'
summary_df.at[1, 'type'] = 'details'
summary_df.at[2, 'type'] = 'description'

# summary_df

In [30]:
intro_df = source_df.loc[INTRO_ROWS].reset_index(drop=True)
# intro_df

In [31]:
# Combine the rows as needed so that:
# row 0 is the Intro Header
# row 1 is the Intro Details

for col in intro_df.columns:
    intro_df.at[1, col] = intro_df.at[1, col] + ' ' + intro_df.at[2, col]

intro_df = intro_df.drop(2)

intro_df

,en,es,zh-tw,hy,ko,vi,ja,ru
0,Metro is restoring full service levels.,Metro está restaurando los niveles completos de servicio.,Metro將對巴士服務進行變更，以恢復整體的服務水準。,Metro-ն փոխում է ավտոբուսների սպասարկումը՝ վերականգնելու սպասարկման ամբողջական մակարդակները:,Metro는 전체 서비스 수준을 복구하기 위해 버스편 운행 변경을 진행하고 있습니다.,Metro sắp tiến hành thay đổi dịch vụ xe buýt để khôi phục mức dịch vụ đầy đủ.,Metroは、完全なサービスレベルに復旧するために、バス運行サービスの改正を実施いたします。,Metro вносит ряд изменений в график движения автобусов для возобновления полного обслуживания.
1,"New schedules beginning December 11, 2022. Metro is restoring full service levels, which were reduced in early 2022 due to operator shortages. Service changes\nto affected bus lines will begin Sunday, December 11. Check specific bus line schedules at metro.net/mybus.",Los nuevos horarios comienzan el 11 de deciembre 2022. Metro está restaurando los niveles de servicio completo que se redujeron a principios de 2022 debido a la falta de operadores. Los cambios en el servicio de las líneas de autobús afectadas comenzarán el domingo 11 de diciembre. Consulte los horarios de las líneas de autobús específicas en metro.net/mybus.,最新班次時間表將於2022年12月11日生效。 Metro即將在12月11日週日更動其巴士線路，藉此向搭乘者提供更多班次與更加穩定的服務。此服務所進行的變更，使得所有在2022年年初縮減的整體服務水準得以全面恢復。欲知更多詳情，請至metro.net/mybus查看詳細的巴士線路時間表。,"Նոր ժամանակացույցեր՝ սկսած 2022 թ-ի դեկտեմբերի 11-ից: Metro-ն կիրակի օրը՝ դեկտեմբերի 11-ից սկսած, որոշ երթուղիների գծերում փոփոխություններ է կատարում՝ մեր ուղևորներին ավելի հաճախակի և հուսալի ծառայություններ մատուցելու համար: Ծառայությունների այս փոփոխությունն ավարտում է 2022 թվականին ավելի վաղ նվազեցված սպասարկման ամբողջական մակարդակների վերականգնումը: Լրացուցիչ մանրամասների համար այցելեք metro.net/mybus կայքը՝ ավտոբուսային գծի հատուկ ժամանակացույցերի հատվածը:",2022년 12월 11일부터 새로운 일정이 적용됩니다. Metro는 승객들에게 안정적으로 더 많은 서비스를 제공하기 위해 12월 11일 일요일부터 일부 버스 노선의 운행 서비스 변경을 진행하고 있습니다. 이번 서비스 변경으로 2022년 초에 축소되었던 전체 서비스 수준의 복구가 완료됩니다. 자세한 내용은 metro.net/mybus에서 구체적인 버스 노선 일정을 확인하십시오.,"Lịch trình mới sẽ bắt đầu từ ngày 11 tháng 12 năm 2022. Metro sắp tiến hành thay đổi dịch vụ đối với một số tuyến xe buýt kể từ Chủ Nhật, ngày 11 tháng 12, để cung cấp cho hành khách các dịch vụ thường xuyên và đáng tin cậy hơn. Sự thay đổi dịch vụ này sẽ hoàn thành việc khôi phục mức dịch vụ đầy đủ mà đã trước đó đã bị giảm đi hồi năm 2022. Để biết thêm thông tin chi tiết, hãy xem lịch trình cụ thể của các tuyến xe buýt tại metro.net/mybus.",新しいダイヤは、2022年12月11日より運行開始となります。 Metroは、乗客の皆様に向けて運行本数の多い信頼できるサービスを提供するべく、12月11日（日）より一部の路線において運行サービスの改正を実施いたします。この改正により、2022年のある時点より低下しておりましたサービスレベルが完全なレベルに復旧します。詳細につきましては、metro.net/mybusにて路線別時刻表をご覧ください。,"Новый график будет действовать с 11 декабря 2022 г. С воскресенья, 11 декабря, вступают в силу изменения в графике движения некоторых автобусных маршрутов Metro. Это позволит сделать наши рейсы более частыми, а перевозки — более надежными и завершить восстановление полного уровня обслуживания, представленного в начале 2022 года. Чтобы узнать больше, ознакомьтесь с графиком движения конкретных автобусных маршрутов на странице metro.net/mybus."


In [32]:
# def remove_sentence(row):
#     if row.name == 4:
#         for col in intro_df.columns:
#             if ':' in row[col]:
#                 intro_df.at[4, col] = row[col].split(':')[0].replace('"', '') + ':'
#             elif '。' in row[col]:
#                 intro_df.at[4, col] = row[col].split('。')[0] + '。'
#             else:
#                 if col == 'ru':
#                     intro_df.at[4, col] = row[col].split('.')[0] + '. ' + row[col].split('.')[1] + '.'
#                 else:
#                     intro_df.at[4, col] = row[col].split('.')[0] + '. '
#     return

# intro_df.apply(remove_sentence, axis=1)

# intro_df = intro_df.reset_index(drop=True)

In [33]:
#set an array for the file types
language_list = {
    'English': {
        'label': 'English',
        'order': 0
    },
    'Spanish': {
        'label': 'Español (Spanish)',
        'order': 1
    },
    'Chinese': {
        'label': '中文 (Chinese Traditional)',
        'order': 2
    },
    'Korean': {
        'label': '한국어 (Korean)',
        'order': 3
    },
    'Vietnamese': {
        'label': 'Tiếng Việt (Vietnamese)',
        'order': 4
    },
    'Japanese': {
        'label': '日本語 (Japanese)',
        'order': 5
    },
    'Russian': {
        'label': 'русский (Russian)',
        'order': 6
    },
    'Armenian': {
        'label': 'Армянский (Armenian)',
        'order': 7
    }
}
    
takeones_list = {}


In [34]:

#create a list of file types
for root, dirs, files in os.walk("../files/takeones"):
    for filename in files:
        lang = filename.split('_-_')[1].split('.')[0]
        takeones_list[language_list[lang]['order']] = {
            'text': language_list[lang]['label'],
            'url': '/files/takeones/' + filename
        }
        
# takeones_list


In [35]:

takeones_dict = {
    'takeones': {
        'description': 'Download a PDF version of this page in ',
        'files': takeones_list
        }
}

In [36]:
# temp = summary_df[['type','en']]
# temp = temp.rename(columns={'en': 'description'})

# for i in range(len(temp['description'])):
#     if temp.loc[i, 'type'] == 'details':
#         temp.loc[i-1, 'details'] = temp.loc[i, 'description']
#         temp = temp.drop(i)

# temp = temp.reset_index(drop=True)
# temp = temp.drop('type', axis=1)
# temp

In [37]:
# details_df.to_json('lineChangesOctober.json', orient='records')

details_df = details_df.fillna('')
langs = ['en','es','zh-tw','ko','vi','ja','ru','hy']
all_langs_results_arr = []

for lang in langs:
    ### Intro Section
    temp_intro_df = intro_df[lang]
    temp_intro_df = temp_intro_df.rename(index={0: 'header', 1: 'details'})

    temp_intro_dict = temp_intro_df.to_dict()
    temp_intro_dict = {
        'intro': temp_intro_dict
    }

    ### Summary Section
    temp_summary_df = summary_df[['type',lang]]
    temp_summary_df = temp_summary_df.rename(columns={lang: 'description'})

    for i in range(len(temp_summary_df['description'])):
        if temp_summary_df.loc[i, 'type'] == 'details':
            temp_summary_df.loc[i-1, 'details'] = temp_summary_df.loc[i, 'description']
            temp_summary_df = temp_summary_df.drop(i)
        # if i % 2 == 0:
        #     temp_summary_df.loc[i, 'details'] = temp_summary_df.loc[i+1, 'description']
        # else:
        #     temp_summary_df = temp_summary_df.drop(i)
    temp_summary_df = temp_summary_df.drop('type', axis=1)
    temp_summary_df = temp_summary_df.fillna('')

    temp_summary_dict = temp_summary_df.to_dict(orient='records')
    temp_summary_df = pd.DataFrame({'summary': temp_summary_dict})

    temp_summary_dict = temp_summary_df.to_dict(orient='list')

    ### Details Section
    temp_details_df = details_df[[lang, 'line', 'label', 'newSchedule', 'currentSchedule']]
    temp_details_df = temp_details_df.rename(columns={lang: 'description'})
    temp_details_dict = temp_details_df.to_dict(orient='records')

    temp_details_df = pd.DataFrame({'details': temp_details_dict})
    temp_details_dict = temp_details_df.to_dict(orient='list')

    # use the dictionary unpacking notation: **
    # to create a new dictionary comprised of the key/value pairs from the other dictionaries
    result_dict = temp_intro_dict
    result_dict.update(takeones_dict)
    result_dict.update(temp_summary_dict)
    result_dict.update(temp_details_dict)
    
    all_langs_results_arr.append({
        'lang': lang,
        'content': result_dict
    })

# print(all_langs_results_dict)


In [38]:
import json

# all_langs_results_json = json.dumps(all_langs_results_dict)
# print(all_langs_results_json)

with open('../data/allChanges.json', 'w') as json_file:
    json.dump(all_langs_results_arr, json_file)